<hr><center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks&para;">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective&para;">Objective</h3>
<br><strong style="color: #000000;"><em>Configure url endpoint to the SNS associated with a cloudwatch alarm</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Stop-Untagged-EC2-Instances&para;"><u>Attach a webhook endpoint to AWS Cloudwatch alarm</u></h2>
</center>
<h1 id="Steps-Overview&para;">Steps Overview</h1>
<p>1)&nbsp;<a href="#1">Attach a webhook endpoint to AWS Cloudwatch alarm</a></p>

<h3 id="Get-all-Untagged-EC2-Instances"><a id="2" target="_self" rel="nofollow"></a>Attach a webhook endpoint to AWS Cloudwatch alarms</h3>
<p>Here we will configure the url endpoint to the SNS associated with a cloudwatch alarm.&nbsp;</p>
<blockquote>
<p>This action takes the following parameters: <code>alarm_name, region, region</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>None</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

from pydantic import BaseModel, Field
from unskript.legos.aws.aws_get_handle.aws_get_handle import Session
from urllib.parse import urlparse
import pprint

from beartype import beartype
@beartype
def aws_cloudwatch_attach_webhook_notification_to_alarm_printer(output):
    if output is None:
        return
    pprint.pprint({"Subscription ARN" : output})


@beartype
def aws_cloudwatch_attach_webhook_notification_to_alarm(
    hdl: Session,
    alarm_name: str,
    region: str,
    url: str
) -> str:
    """aws_cloudwatch_attach_webhook_notification_to_alarm returns subscriptionArn

        :type alarm_name: string
        :param alarm_name: Cloudwatch alarm name.

        :type url: string
        :param url: URL where the alarm notification needs to be sent.

        :type region: string
        :param region: AWS Region of the cloudwatch.

        :rtype: Returns subscriptionArn
    """
    cloudwatchClient = hdl.client("cloudwatch", region_name=region)

    # Get the configured SNS(es) to this alarm.
    alarmDetail = cloudwatchClient.describe_alarms(
        AlarmNames=[alarm_name]
    )
    if alarmDetail is None:
        return f'Alarm {alarm_name} not found in AWS region {region}'
    # Need to get the AlarmActions from either composite or metric field.
    if len(alarmDetail['CompositeAlarms']) > 0:
        snses = alarmDetail['CompositeAlarms'][0]['AlarmActions']
    else:
        snses = alarmDetail['MetricAlarms'][0]['AlarmActions']

    #Pick any sns to configure the url endpoint.
    if len(snses) == 0:
        return f'No SNS configured for alarm {alarm_name}'

    snsArn = snses[0]
    print(f'Configuring url endpoint on SNS {snsArn}')

    snsClient = hdl.client('sns', region_name=region)
    # Figure out the protocol from the url
    try:
        parsedURL = urlparse(url)
    except Exception as e:
        print(f'Invalid URL {url}, {e}')
        raise e

    if parsedURL.scheme != 'http' and parsedURL.scheme != 'https':
        return f'Invalid URL {url}'

    protocol = parsedURL.scheme
    try:
       response = snsClient.subscribe(
            TopicArn=snsArn,
            Protocol=protocol,
            Endpoint=url,
            ReturnSubscriptionArn=True)
    except Exception as e:
        print(f'Subscribe to SNS topic arn {snsArn} failed, {e}')
        raise e
    subscriptionArn = response['SubscriptionArn']
    print(f'URL {url} subscribed to SNS {snsArn}, subscription ARN {subscriptionArn}')
    return subscriptionArn



task = Task(Workflow())
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_cloudwatch_attach_webhook_notification_to_alarm, lego_printer=aws_cloudwatch_attach_webhook_notification_to_alarm_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we demonstrated the use of unSkript's AWS legos to configure the url endpoint to the SNS associated with a cloudwatch alarm. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>